In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import json
from pandas import json_normalize
import os
Home="/content/drive/MyDrive/aware_train_2022"
classes_score=json.load(open(os.path.join(Home, "class_scores.json")))
#dummy_run=json.load(open(os.path.join(Home, "dummy_run.json"))) # delimiter error
gt_train=json.load(open(os.path.join(Home, "gt_train.json")))
gt_val=json.load(open(os.path.join(Home, "gt_val.json")))
pred_test=json.load(open(os.path.join(Home, "predictions_test.json")))
pred_train=json.load(open(os.path.join(Home, "predictions_train.json")))
pred_val=json.load(open(os.path.join(Home, "predictions_val.json")))

In [ ]:
#classes_score format VC{situation[4]}
print("Classes_Scores")
print("total number of scenarios",len(classes_score))


Classes_Scores
total number of scenarios 266


In [ ]:
#gt_train format userid{situation[4]}
print("Actual Train Outputs")
print("total number of userids in out Trainset",len(gt_train))

Actual Train Outputs
total number of userids in out Trainset 600


In [ ]:
#gt_val format userid{situation[4]}
print("Actual validation Outputs")
print("total number of userids in out val",len(gt_val))


Actual validation Outputs
total number of userids in out val 200


In [ ]:
#pred_train format userid{ userimg {"VC", confidence, points[4]}
print("Actual Train Input")
print("total number of userids in Trainset",len(pred_train))
maxVC=0
pred={"Train":pred_train,"val":pred_val,"test":pred_test}
for s,p in pred.items():
  maxVC=0
  for user in p:
    for img in p[user]:
      maxVC=max(maxVC,len(p[user][img]))
  print("number of VC in ",s,maxVC)

for s,p in pred.items():
  maxIMG=0
  for user in p:
    maxIMG=max(maxIMG,len(p[user]))
  print("number of images in ",s,maxIMG)

Actual Train Input
total number of userids in Trainset 600
number of VC in  Train 20
number of VC in  val 17
number of VC in  test 18
number of images in  Train 93
number of images in  val 92
number of images in  test 94


In [ ]:
#pred_val format userid{ userimg {"VC", confidence, points[4]}
print("Actual Val Input")
print("total number of userids in val",len(pred_val))
print(pred_val["zjn7scaggcdjod7u"]["l9ubiwvo7sqes0ft"][0])

Actual Val Input
total number of userids in val 200
VC62 0.348204642534256 0.0 0.16885580122470856 1.0 0.3646084666252136


In [ ]:
print("max number of images for each user , pred_train",max([len(user) for user in pred_train]))
print("max number of images for each user, pred_val ",max([len(user) for user in pred_val]))
print("max number of images for each user, pred_val ",max([len(user) for user in pred_val]))


max number of images for each user , pred_train 16
max number of images for each user, pred_val  16
max number of images for each user, pred_val  16


In [ ]:
import numpy
import array
import tensorflow as tf



def getmatrix(user,pred): #for flattened 9columns, append each image { visual_concept{acc,it, bank,wait},confidence,4 points} 
  images=[img for img in pred[user]]
  matrix=[]
  for img in images:
    VCs=[s for s in pred[user][img]] #gets the strings{VC,confidence,4 points} inside image
    for VC in VCs:
      row=VC.split(" ")
      VC_name= row[0]
      scores=[]
      try: #some VC not found in class_score
        scores=[float(classes_score[VC_name]["acc"]),float(classes_score[VC_name]["it"]),float(classes_score[VC_name]["bank"]),float(classes_score[VC_name]["wait"])] #[it for k2,it in classes_score[row[0]].items()]
      except:
        scores=[-1,-1,-1,-1]
      scores=[]
      for i in range(1,2): #if this is 2,3,4,5 are the location coords
        scores.append(float(row[i]))
      matrix.extend(scores[:])
  while(len(matrix)<=3000):
    matrix.extend([-1]*1)
  flatten_list = matrix
  return flatten_list

x_train=[getmatrix(user,pred_train) for user in pred_train]
y_train=[]
y_train1,y_train2,y_train3,y_train4=[],[],[],[]
for user in gt_train:
  y_train.append([float(gt_train[user]["acc"]),float(gt_train[user]["it"]),float(gt_train[user]["bank"]),float(gt_train[user]["wait"])])


x_val=[getmatrix(user,pred_val) for user in pred_val]

y_val=[]
for user in gt_val:
  y_val.append([float(gt_val[user]["acc"]),float(gt_val[user]["it"]),float(gt_val[user]["bank"]),float(gt_val[user]["wait"])])

print("Fit model on training data")

from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import *
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
models = {"ETreeR":ExtraTreesRegressor,
          "RForestR":RandomForestRegressor,
          "GBR":GradientBoostingRegressor}
val = {"ETreeR":[0.5010044342130072,7.023577170121131e-16],"RForestR":[0.498759369775145,0.17990357853684089],"GBR":[0.5037543503228156, 0.1806476008687124]}
def save_result(result,name):
  dictdata={}
  for row,user in zip(result,pred_test):
    dictdata[user]={}
    dictdata[user]["acc"]=float(row[0])
    dictdata[user]["bank"] = float(row[1])
    dictdata[user]["it"] = float(row[2])
    dictdata[user]["wait"] = float(row[3])
  with open('/content/drive/MyDrive/aware_train_2022/output/'+name, 'w') as f:
      json.dump(dictdata, f)
print(val)
for n in models.keys():
  MOR=MultiOutputRegressor(models[n](random_state=0)).fit(x_train, y_train)
  print(n)
  val_err = mean_absolute_error(y_val,MOR.predict(x_val),squared=False)
  err = mean_absolute_error(y_train,MOR.predict(x_train),squared=False)
  print(val_err,",",err)
  if(err<val[n][1] and val_err <val[n][0]):
    print("changed",n)
    val[n][1] = err
    val[n][0] = val_err
    x_test=[getmatrix(user,pred_test) for user in pred_test]
    result_mor=MOR.predict(x_test)
    #save_result(result_mor,n+'.json')
print(val)
  



Fit model on training data
{'ETreeR': [0.5010044342130072, 7.023577170121131e-16], 'RForestR': [0.498759369775145, 0.17990357853684089], 'GBR': [0.5037543503228156, 0.1806476008687124]}
ETreeR
0.40270373673080334 , 4.705694023101244e-16
changed ETreeR
RForestR
0.4040859703900409 , 0.14336548257278603
changed RForestR
GBR
0.42080292842710815 , 0.20221363104174547
{'ETreeR': [0.40270373673080334, 4.705694023101244e-16], 'RForestR': [0.4040859703900409, 0.14336548257278603], 'GBR': [0.5037543503228156, 0.1806476008687124]}


In [ ]:
print(7.023577170121131e-16 < 7.023577170121131e-15)

True


In [ ]:
x_test=[getmatrix(user,pred_test) for user in pred_test]
result_mor=MOR.predict(x_test)
def save_result(result,name):
  dictdata={}
  for row,user in zip(result,pred_test):
    dictdata[user]={}
    dictdata[user]["acc"]=str(row[0])
    dictdata[user]["bank"] = str(row[1])
    dictdata[user]["it"] = str(row[2])
    dictdata[user]["wait"] = str(row[3])
  with open('/content/drive/MyDrive/aware_train_2022/output/'+name, 'w') as f:
      json.dump(dictdata, f)
save_result(result_mor,"RForestR.json")

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K




def getmatrix(user,pred): #for flattened 9columns, append each image { visual_concept{acc,it, bank,wait},confidence,4 points} 
  images=[img for img in pred[user]]
  matrix=[]
  for img in images:
    VCs=[s for s in pred[user][img]] #gets the strings{VC,confidence,4 points} inside image
    for VC in VCs:
      row=VC.split(" ")
      VC_name= row[0]
      scores=[]
      try: #some VC not found in class_score
        scores=[float(classes_score[VC_name]["acc"]),float(classes_score[VC_name]["it"]),float(classes_score[VC_name]["bank"]),float(classes_score[VC_name]["wait"])] #[it for k2,it in classes_score[row[0]].items()]
      except:
        scores=[-1,-1,-1,-1]
      # for i in range(1,2):
      #   scores.append(float(row[i]))
      matrix.extend(scores[:])
  while(len(matrix)<=3000):
    matrix.extend([-1]*4)
  flatten_list = matrix
  return flatten_list

x_train=[getmatrix(user,pred_train) for user in pred_train]
y_train=[]
y_train1,y_train2,y_train3,y_train4=[],[],[],[]
for user in gt_train:
  y_train.append([float(gt_train[user]["acc"]),float(gt_train[user]["it"]),float(gt_train[user]["bank"]),float(gt_train[user]["wait"])])


x_val=[getmatrix(user,pred_val) for user in pred_val]

y_val=[]
for user in gt_val:
  y_val.append([float(gt_val[user]["acc"]),float(gt_val[user]["it"]),float(gt_val[user]["bank"]),float(gt_val[user]["wait"])])



DenseModel = Sequential([
    Dense(3000, activation='relu'),
    Dense(2048, activation='relu'),
     Dense(1024, activation='relu'),
     Dense(512, activation='relu'),
     Dense(128, activation='relu'),
     Dense(64, activation='relu'),
    Dense(4)
])

DenseModel.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=Adam(),
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

DenseModel.fit(x_train, y_train, epochs=200,validation_data=(x_val,y_val),batch_size=20)
model=DenseModel
print("Validation error: ",mean_squared_error(y_val,model.predict(x_val),squared=False))
print("Train Error",mean_squared_error(y_train,model.predict(x_train),squared=False))

Epoch 1/200
30/30 [==============================] - 3s 100ms/step - loss: 3.3604 - root_mean_squared_error: 1.8331 - val_loss: 0.4170 - val_root_mean_squared_error: 0.6457
Epoch 2/200
30/30 [==============================] - 0s 8ms/step - loss: 0.3397 - root_mean_squared_error: 0.5829 - val_loss: 0.2970 - val_root_mean_squared_error: 0.5450
Epoch 3/200
30/30 [==============================] - 0s 8ms/step - loss: 0.2696 - root_mean_squared_error: 0.5192 - val_loss: 0.3385 - val_root_mean_squared_error: 0.5818
Epoch 4/200
30/30 [==============================] - 0s 8ms/step - loss: 0.2618 - root_mean_squared_error: 0.5117 - val_loss: 0.2791 - val_root_mean_squared_error: 0.5283
Epoch 5/200
30/30 [==============================] - 0s 7ms/step - loss: 0.2651 - root_mean_squared_error: 0.5148 - val_loss: 0.2839 - val_root_mean_squared_error: 0.5329
Epoch 6/200
30/30 [==============================] - 0s 7ms/step - loss: 0.2606 - root_mean_squared_error: 0.5105 - val_loss: 0.2640 - val_root

KeyboardInterrupt: ignored

In [ ]:
DenseModel.save('/content/drive/MyDrive/aware_train_2022/models/DMbest')
x_test=[getmatrix(user,pred_test) for user in pred_test]
result_Dense=DenseModel.predict(x_test)
def save_result(result,name):
  dictdata={}
  for row,user in zip(result,pred_test):
    dictdata[user]={}
    dictdata[user]["acc"]=str(row[0])
    dictdata[user]["bank"] = str(row[1])
    dictdata[user]["it"] = str(row[2])
    dictdata[user]["wait"] = str(row[3])
  with open('/content/drive/MyDrive/aware_train_2022/output/'+name, 'w') as f:
      json.dump(dictdata, f)
save_result(result_Dense,"DenseModel.json")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/aware_train_2022/models/DMbest/assets


In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras import datasets, layers, models



def getmatrix(user,pred): #for flattened 9columns, append each image { visual_concept{acc,it, bank,wait},confidence,4 points} 
  images=[img for img in pred[user]]
  matrix=[]
  for img in images:
    VCs=[s for s in pred[user][img]] #gets the strings{VC,confidence,4 points} inside image
    for VC in VCs:
      row=VC.split(" ")
      VC_name= row[0]
      scores=[]
      try: #some VC not found in class_score
        scores=[float(classes_score[VC_name]["acc"]),float(classes_score[VC_name]["it"]),float(classes_score[VC_name]["bank"]),float(classes_score[VC_name]["wait"])] #[it for k2,it in classes_score[row[0]].items()]
      except:
        scores=[-1,-1,-1,-1]
      for i in range(1,2):
        scores.append(float(row[i]))
      matrix.append(scores[:])
  while(len(matrix)<=300):
    matrix.append([-1]*5)
  flatten_list = matrix

  return flatten_list

x_train=[getmatrix(user,pred_train) for user in pred_train]
y_train=[]
y_train1,y_train2,y_train3,y_train4=[],[],[],[]
for user in gt_train:
  y_train.append([float(gt_train[user]["acc"]),float(gt_train[user]["it"]),float(gt_train[user]["bank"]),float(gt_train[user]["wait"])])


x_val=[getmatrix(user,pred_val) for user in pred_val]

y_val=[]
for user in gt_val:
  y_val.append([float(gt_val[user]["acc"]),float(gt_val[user]["it"]),float(gt_val[user]["bank"]),float(gt_val[user]["wait"])])


model = models.Sequential()
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(4))


model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=Adam(),
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

model.fit(x_train, y_train, epochs=100,validation_data=(x_val,y_val),batch_size=40)
print("Validation error: ",mean_squared_error(y_val,model.predict(x_val),squared=False))
print("Train Error",mean_squared_error(y_train,model.predict(x_train),squared=False))

Epoch 1/100
15/15 [==============================] - 2s 125ms/step - loss: 0.9296 - root_mean_squared_error: 0.9641 - val_loss: 0.3664 - val_root_mean_squared_error: 0.6053
Epoch 2/100
15/15 [==============================] - 0s 6ms/step - loss: 0.3037 - root_mean_squared_error: 0.5511 - val_loss: 0.3080 - val_root_mean_squared_error: 0.5549
Epoch 3/100
15/15 [==============================] - 0s 6ms/step - loss: 0.2532 - root_mean_squared_error: 0.5031 - val_loss: 0.2804 - val_root_mean_squared_error: 0.5295
Epoch 4/100
15/15 [==============================] - 0s 6ms/step - loss: 0.2305 - root_mean_squared_error: 0.4801 - val_loss: 0.2741 - val_root_mean_squared_error: 0.5236
Epoch 5/100
15/15 [==============================] - 0s 5ms/step - loss: 0.2167 - root_mean_squared_error: 0.4655 - val_loss: 0.2724 - val_root_mean_squared_error: 0.5219
Epoch 6/100
15/15 [==============================] - 0s 6ms/step - loss: 0.2040 - root_mean_squared_error: 0.4517 - val_loss: 0.2721 - val_root